# Basic Loading of the Kernel


To run the notebooks we recommend using Poetry and starting a shell with a virtual environment
prepared to use SK.

See [DEV_SETUP.md](../../python/DEV_SETUP.md) for more information.


In [1]:
!python -m pip install semantic-kernel==0.9.7b1

In [2]:
from semantic_kernel import Kernel, kernel

kernel = Kernel()

When using the kernel for AI requests, the kernel needs some settings like URL and credentials to the AI models.

The SDK currently supports OpenAI and Azure OpenAI, among other connectors.

If you need an Azure OpenAI key, go [here](https://learn.microsoft.com/en-us/azure/cognitive-services/openai/quickstart?pivots=rest-api).


In [4]:
from services import Service

# Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
selectedService = Service.OpenAI

In [5]:
service_id = None
if selectedService == Service.OpenAI:
    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
    from semantic_kernel.utils.settings import openai_settings_from_dot_env

    api_key, org_id = openai_settings_from_dot_env()
    service_id = "oai_chat_gpt"
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id, ai_model_id="gpt-3.5-turbo-1106", api_key=api_key, org_id=org_id),
    )
elif selectedService == Service.AzureOpenAI:
    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
    from semantic_kernel.utils.settings import azure_openai_settings_from_dot_env

    deployment, api_key, endpoint = azure_openai_settings_from_dot_env()
    service_id = "aoai_chat_completion"
    kernel.add_service(
        AzureChatCompletion(service_id=service_id, deployment_name=deployment, endpoint=endpoint, api_key=api_key),
    )

Great, now that you're familiar with setting up the Semantic Kernel, let's see [how we can use it to run prompts](02-running-prompts-from-file.ipynb).
